In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('./data/casting.csv', encoding='cp949', index_col=0)

/var/folders/4_/kbkdg_cn0z5_y3bg5394y50h0000gn/T/ipykernel_63897/446435080.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/casting.csv', encoding='cp949', index_col=0)


In [8]:
df = data

In [9]:
df['registration_time'] = pd.to_datetime(df['registration_time'])
df['date'] = df['registration_time'].dt.date
df['hour'] = df['registration_time'].dt.hour
df['weekday'] = df['registration_time'].dt.weekday ## 0:월요일, 6:일요일
df['date_time'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['hour'].astype(str) + ':00:00')
df['weekday'] = df['weekday'].map({0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'})
# 평균 생산시간 계산
df['average_cycle_time'] = (df['facility_operation_cycleTime'] + df['production_cycletime']) / 2
## 정상품, 불량품 생성
df['pass'] = df['passorfail'].apply(lambda x: 1 if x == 0 else 0)
df['fail'] = df['passorfail'].apply(lambda x: 1 if x == 1 else 0)


# 데이터프레임에서 평균 생산 시간과 생산량 계산
grouped_data = df.groupby(['date_time','mold_code','weekday','hour'])['average_cycle_time'].agg(['mean','median','count']).reset_index()
## 불량률 계산
grouped_data2 = df.groupby(['date_time','mold_code','weekday','hour'])['pass'].sum().reset_index(name='pass_count')
grouped_data3 = df.groupby(['date_time','mold_code','weekday','hour'])['fail'].sum().reset_index(name='error_count')

merge_grouped_df = pd.merge(grouped_data, grouped_data2,
                            on=['date_time','mold_code','weekday','hour'], how='left') 

merge_grouped_df = pd.merge(merge_grouped_df, grouped_data3,
                                on=['date_time','mold_code','weekday','hour'], how='left')


In [10]:
merge_grouped_df

,date_time,mold_code,weekday,hour,mean,median,count,pass_count,error_count
0,2019-01-02 16:00:00,8412,수,16,115.937500,112.25,8,8,0
1,2019-01-02 16:00:00,8722,수,16,119.500000,119.50,1,1,0
2,2019-01-02 17:00:00,8412,수,17,113.625000,111.50,28,28,0
3,2019-01-02 17:00:00,8722,수,17,125.000000,122.50,16,0,16
4,2019-01-02 18:00:00,8412,수,18,114.592593,112.00,27,27,0
...,...,...,...,...,...,...,...,...,...
3566,2019-03-30 06:00:00,8917,토,6,126.434783,122.50,23,23,0
3567,2019-03-31 20:00:00,8917,일,20,134.617647,132.50,17,9,8
3568,2019-03-31 21:00:00,8917,일,21,124.321429,121.25,14,11,3
3569,2019-03-31 22:00:00,8917,일,22,122.300000,120.50,15,8,7


In [12]:
data[data['date_time'] == '2019-01-02 16:00:00']

,line,name,mold_name,time,date,count,working,emergency_stop,molten_temp,facility_operation_cycleTime,...,passorfail,tryshot_signal,mold_code,heating_furnace,hour,weekday,date_time,average_cycle_time,pass,fail
0,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,258,가동,ON,731.0,119,...,0.0,NaN,8722,NaN,16,수,2019-01-02 16:00:00,119.5,1,0
1,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,243,가동,ON,720.0,98,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,111.5,1,0
2,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,244,가동,ON,721.0,98,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,110.0,1,0
3,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,245,가동,ON,721.0,100,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,112.5,1,0
4,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,246,가동,ON,721.0,99,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,111.0,1,0
5,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,247,가동,ON,722.0,100,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,112.5,1,0
6,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,248,가동,ON,723.0,104,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,116.5,1,0
7,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,249,가동,ON,723.0,101,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,141.5,1,0
8,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,2019-01-02,250,가동,ON,723.0,102,...,0.0,NaN,8412,B,16,수,2019-01-02 16:00:00,112.0,1,0
